# Assignment 3 - Targeting areas

In [2]:
# allows plot outputs to appear inline
%matplotlib inline

## Imports
# statistical data visualisation
import seaborn as sns
# general data analysis
import pandas as pd
# spatial data analysis
import pysal as ps
# spatial data manipulation and visualisation
import geopandas as gpd
# mathematical functions and arrays
import numpy as np
# general visualisation utilities
import matplotlib.pyplot as plt
# use operating system dependent functions, e.g. working directory
import os     
# regular Expressions, search for a pattern in strings
import re
# find all pathnames in a directory that match a specific extension
import glob
# add text and shape effects such as shadows
import matplotlib.patheffects as path_effects
# retrieve and write to disk tile maps from the internet into geospatial raster files, used for basemaps
import contextily as ctx
# allows point geometry to be added to coordinates
from shapely.geometry import Point
# read html data and convert into just text
from bs4 import BeautifulSoup
# used for DBSCAN clustering
from sklearn.cluster import dbscan
# used for KMeans clustering
from sklearn.cluster import KMeans
# allows inset figures
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# able to change values from one crs to another
from pyproj import Proj, transform
# find the distance between two inputs
from scipy.spatial.distance import cdist

# get rid of warnings (only for submission)
import warnings; warnings.simplefilter('ignore')

## Introduction

The focus of this analysis is to determine the potential for infrastructural
improvements in Liverpool that may lead to regeneration, and with that the
potential for reduced crime (Thomson, Atkinson, Petticrew, & Kearns, 2006)⁠⁠, in
areas that are deemed to be crime hotspots, and have shown the potential for an
increase in popularity.

## Data

### Liverpool LSOA Polygons

In [ ]:
# Define path to LSOA shapefile
lsoas_path = './data/liverpool/shapefiles/Liverpool_lsoa11.shp'
# Read polygons into a geopandas.geodataframe, set the index to LSOA codes
lsoas = gpd.read_file(lsoas_path).set_index('LSOA11CD')

In [ ]:
# Define path to names points (Liverpool City centre, Edge Hill, etc.)
names_path = './data/names/NamedPlace.shp'
# Read into geopandas.geodataframe
names = gpd.read_file(names_path)

In [ ]:
 # Select only the Liverpool City Centre point
cityCentre = names.loc[names['distname'] == 'LIVERPOOL']

### Liverpool Postcodes Coordinate Data

In [ ]:
## Coordinate data for all Liverpool Postcodes into geopandas point geodataframe ##
# Read North West postcode data (huge file)
NWPC = pd.read_csv('./data/NWPC.csv', index_col = 'Postcode')

# Combine Easting and Northing to one column called geometry
NWPC['geometry'] = list(zip(NWPC.Easting, NWPC.Northing))
# Add point geometry
NWPC['geometry'] = NWPC['geometry'].apply(Point)
# Set crs to british national grid
NWPC.crs = {'init': 'epsg:27700'}
# Turn into a geopandas.geodataframe
NWPC = gpd.GeoDataFrame(NWPC, crs=NWPC.crs, geometry=NWPC['geometry'])

# Select only LSOAs from the liverpool local authority district code
# and rename to lpc (Liverpool Postcodes)
lpc = NWPC.loc[NWPC['District Code'] == 'E08000012']
# Remove useless columns
lpc = NWPC.iloc[:,[3,4,9]]
# (Re)Append the geometry
lpc['geometry'] = NWPC[['geometry']]

### Recent Planning Descisions (past 60 days)

In [ ]:
## Web pages downloaded as python couldn't strip the data, 66 in total
## Was able to use urllib but couldn't figure out how to generate a search query through it :(

s = "" # define s as an empty string for later

# Define path and file extension
path = './data/planninghtml/*.html'   
# Glob lists all files with .html extension in path
files = glob.glob(path)

# For loop to read each html file in the directory
for name in files:
    # Define name when opened as f, 'with' closes file when read (avoids problems with files left open)
    with open(name) as f:
        # Beautiful soup can pull the html data to convert to text
        soup = BeautifulSoup(f)
        # Convert into one large string from each .html
        s += soup.get_text()

In [ ]:
## Set up Planning Postcodes dataframe ##
## Regex from https://en.wikipedia.org/wiki/Postcodes_in_the_United_Kingdom#Validation
## Finds all valid postcodes contained within string 's' (regular expression)
ppc = re.findall(r'[A-Z]{1,2}[0-9R][0-9A-Z]? [0-9][A-Z]{2}', s)

# Convert from string into pandas dataframe
ppc = pd.DataFrame(ppc)
# Rename first column to postcode
ppc.columns = ['postcode']
# Set postcode column as index
ppc = ppc.set_index('postcode')
# Add a Planning Application counts column for combining all Liverpool postcodes later
ppc['planning'] = 1

In [ ]:
# Join planning postcodes to all Liverpool postcodes
ppc = lpc.join(ppc)
# Drop NA values (postcodes with no recent planning decisions)
ppc = ppc.dropna()

In [ ]:
## Find planning application counts per LSOA ##
# Right spatial join Postcodes with planning counts to Liverpool LSOAs
# Right join uses keys from ppc, retains only LSOA geometry
join = gpd.sjoin(ppc, lsoas, how='right')
# Remove columns not needed
join = join.drop(['Easting', 'Northing', 'District Code'], axis=1)
# Rename index to LSOA11CD to merge with LSOA geometry later
join.index.names = ['LSOA11CD']
# Groupby combines all duplicate LSOA and sums postcode counts, rename df to planning
planning = join.groupby(level=0).sum()

### House Sales

In [ ]:
# Define path to house price shapefile
hp = './data/houseprice/shapefiles/E08000012_all.shp'
# Read into geopandas and set index as LSOAs
hp = gpd.read_file(hp).set_index('LSOA11C')
# Rename index to be consistent
hp.index.names = ['LSOA11CD']
# Change NAs to 0 (NA is LSOA with no houses sold)
hp = hp.fillna(0)

In [ ]:
# Find total number of properties sold 2005 - 2015
Sales1995_2005 = hp['fr_2005'] + hp['fr_2006'] + hp['fr_2007'] + hp['fr_2009'] + hp['fr_2010'] + \
                      hp['fr_2011'] + hp['fr_2012'] + hp['fr_2013']+ hp['fr_2015'] + hp['fr_2015']

# Find total number of properties sold 1995 - 2005
Sales2005_2015 =   hp['fr_1995'] + hp['fr_1996'] + hp['fr_1997'] + hp['fr_1999'] + hp['fr_2000'] + \
                      hp['fr_2001'] + hp['fr_2002'] + hp['fr_2003']+ hp['fr_2005'] + hp['fr_2005'] 

# Find the relative percentage change in sales between the two decades
hp['Sales_Change'] = (Sales2005_2015 / Sales1995_2005)*100

# Rename the dataframe to houseSales, remove all other columns
houseSales = hp[['Sales_Change']]

### Public Transport

In [ ]:
## Various data in regards to employment and travel
## Travel time, Destination and Origin indicators of employment locations

# Define path to csv
emp_path = './data/employment.csv'

In [ ]:
# Read the employment transport csv and set index as LSOA
emp = pd.read_csv(emp_path, index_col='EMPLO001')
# Select only LSOA from the liverpool local authority district
emp = emp.loc[emp.EMPLO003 == 'E08000012']

In [ ]:
###
# Travel time, destination and origin indicators to key sites and services
# by Lower Super Output Area (LSOA) (ACS05)
###

# Select: Travel time to nearest employment centre by Public Transport/walk
pTransport = emp[['EMPLO008']]

# Rename EMPLO104 to dist_pt (distance by public transport)
pTransport = pTransport.rename(columns={'EMPLO008': 'dist_pt'})
# Rename index to match lsoa naming
pTransport.index.names = ['LSOA11CD']

### Broadband Speed

In [ ]:
# Define path to broadband speed point shapefile
bb_path = './data/broadband/shapefiles/E08000012.shp'
# Read into a geopandas.geodataframe
bb = gpd.read_file(bb_path)
# Combine <2mb column and <10mb column to make an overall slow broadband column
bb['<10mb'] = bb['num_sl'] + bb['num_av']
# Set index to postcodes
bb = bb.set_index('pcd')
# Set crs as British National Grid
bb.crs = {'init': 'epsg:27700'}
# Turn into a geopandas.geodataframe again
bb = gpd.GeoDataFrame(bb, crs=bb.crs, geometry=bb['geometry'])
# Rename index to postcode
bb.index.names = ['postcode']

In [ ]:
# Right join uses keys from bb, retains only LSOA geometry
broadband = gpd.sjoin(bb, lsoas, how='right')
# Average percentage of homes in each postcode within LSOA that have less than 10mb download speed
broadband = broadband.groupby(level=0).mean()
# Rename index for consistency
broadband.index.names = ['LSOA11CD']
# Define new dataframe just with slow broadband
slowBroadband = broadband[['<10mb']]

### Crime Data

In [ ]:
# Define path and file extension
path = './data/crime/*'   
# Glob lists all files with .csv extension in path, data was provided in individual files
# recursive=True allows glob to search each file
files = glob.glob('./data/crime/**/*.csv', recursive=True)
# Combines all csv in 'files' into one large dataframe (Concatenate)
crime = pd.concat([pd.read_csv(f) for f in files])

# Find non numeric values in dataframe
find = crime.applymap(np.isreal)
# Use Longitude to check there are no headers inside csv (none found, maybe pandas removes them)
find = find.loc[find['Longitude'] == False]

# Subset randomly to 1/10 the dataframe, turns out 180k rows was too ambitious later on
crime = crime.sample(frac=0.1)

In [ ]:
# Set index as LSOA
crime = crime.set_index('LSOA code')
# Remove unecessary columns
crime = crime[['Crime type', 'Longitude', 'Latitude']]
# Rename index for consistency
crime.index.names = ['LSOA11CD']

# Combine Easting and Northing to one column called geometry
crime['geometry'] = list(zip(crime['Longitude'], crime['Latitude']))
# Add point geometry
crime['geometry'] = crime['geometry'].apply(Point)
# Set crs to WGS84 (this is the crs the data comes as)
crime.crs = {'init': 'epsg:4326'}
# Turn into a geopandas geodataframe
crime = gpd.GeoDataFrame(crime, crs=crime.crs, geometry=crime['geometry'])
# Reproject crs as British National Grid
crime = crime.to_crs({'init': 'epsg:27700'})

In [ ]:
## For loop to change the Lat Lon to Easting and Northing for British National Grid ##

# Allows the for loop to numerically iterate through index
crime = pd.DataFrame.reset_index(crime)

# Define current crs values and required crs values
inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:27700')

# Loop for each row to change values, iterrows() required so that index can be used for iteration
for i, row in crime.iterrows():
    # Easting and Northing output for each row, transform from pyproj takes input proj and output proj
    # have to specify the exact location of values so use df.at to do this
    e, n = transform(inProj,outProj, crime.at[i, 'Longitude'], crime.at[i, 'Latitude'])
    
    # Input each new value, again need to use df.at
    crime.at[i, 'Longitude'] = e
    crime.at[i, 'Latitude'] = n
    
# this loop took too long with 180k rows

In [ ]:
# rename from Lat Lon to Easting and Northing
crime.rename(columns={'Longitude':'Easting', 'Latitude': 'Northing'}, inplace=True)
# Lots of points fall outside the lsoa polygons, inner join to find those that fall inside
crime = gpd.sjoin(crime, lsoas, how='inner')
# Remove column created in the join
crime = crime.drop(['index_right'], axis=1)

## Analysis

### Global Spatial Autocorrelation

In [ ]:
## Join all polygon attributes into one dataframe
l = [slowBroadband, houseSales]
dfs = planning.join(pTransport)
# Rename df to spatial lag for analysis
spatialLag = dfs.join(l)

# Add the LSOA geometry by shared column (LSOA names)
spatialLag = spatialLag.merge(lsoas, on='LSOA11CD')

In [ ]:
# Find queen weights for the LSOAs
w_queen = ps.weights.Queen.from_dataframe(spatialLag)
# Row standardise the queen weights (sum of rows add to 1)
w_queen.transform = 'R'

In [ ]:
## Can't seem to iterate Moran's I so did it manually
# Call the Moran's I function for each attribute
mi1 = ps.Moran(spatialLag['planning'], w_queen)
mi2 = ps.Moran(spatialLag['dist_pt'], w_queen)
mi3 = ps.Moran(spatialLag['<10mb'], w_queen)
mi4 = ps.Moran(spatialLag['Sales_Change'], w_queen)

# Create a dataframe to display all observed values
MI = pd.DataFrame({'Planning': [mi1.I, mi1.p_sim],
                   'Distance by Public Transport': [mi2.I, mi2.p_sim],
                   'Slow Broadband': [mi3.I, mi3.p_sim],
                   'House Sales Change': [mi4.I, mi4.p_sim]},
                   index=['Morans I', 'P Values'])
# Round to 3 decimals
MI = np.round(MI, decimals=3)

### Table 1: Morans I for each attribute with associated P values

In [ ]:
MI.T # Show MI values and p values as a table (transformed)

The spatial lag of each attribute was determined, and through Moran’s I used to
assess the level of global spatial autocorrelation for each attribute.

All show
a slight significant positive spatial autocorrelation (Table 1) , however public
transport data appears to give NA, likely due to the data not being strictly
continuous. These results suggest infrastructual improvement to the broadband in
an area may improve neighbouring LSOAs, for example an improvement to an
exchange would improve broadband in all houses connected to that exchange.

### Geodemographic Analysis

Employment and transport data was missing coverage for several LSOAs. A function
was created to allow for missing data imputation, which takes the attribute
means of K Means clusters when NA columns are excluded, and recalculates new
clusters using these values. This takes a common missing data handling technique
(Acock, 2005)⁠⁠, and allows for a more accurate data imputation through K Means,
which normally cannot handle NA values (Mesquita, Gomes, & Rodrigues, 2016)⁠.
Data was also standardised before analysis to ensure attributes are directly
comparable, and to remove the potential for bias weighting during K Means
clustering (Mohamad & Usman, 2013)⁠.

In [ ]:
## Function to standardise all rows within a dataframe ##
## Couldn't get it to ignore geometry properly with 'continue' so only use on regular dataframes ##

def standardise(df):
    """
    Standardise every numeric column in a dataframe.
    All columns must be numeric with no geometry.
    ...
    
    Arguments
    ---------
    df    : pandas.DataFrame
            Dataframe to be standardised
            
    Returns
    -------
    df    : pandas.DataFrame
            Standardised dataframe
    """
    # Loop to iterate through each column
    for column in df:
        # Standardise each column by taking away the mean and dividing by standard dev
        df[column] = df[column] = (df[column] - df[column].mean()) / df[column].std()
    # Return the dataframe
    return df

In [ ]:
## KMeans with Null function ##
## Allows for all columns to be used including columns containing nulls ##
## Works best with many non null columns and when all rows required for analysis ##

def kmeans_withnull(df, num):
    """
    Allows K Means function with null values.
    Substitutes means from non null K Means clusters into dataframe to run.
    Rows in dataframe must all be numeric.
    Created with the ideal that all rows must be maintained, and there are multiple columns with no NA values.
    Must have at least one column with no NA, advised that majority of columns have no NA.
    ...
    
    Arguments
    ---------
    df    : pandas.DataFrame
            Dataframe to run KMeans_withnull analysis
    
    num   : integer
            Number of KMeans clusters
            
    Returns
    -------
    df    : dataframe
            Input dataframe with KMeans cluster labels appended as column
    """
    
    # Take num as the imput for number of clusters
    kmeans = KMeans(n_clusters=num)
    # Save the index of input dataframe for later
    index = df.index.values
    
    # Drop columns with any NA values (Keep all rows)
    dropCol = df.dropna(axis='columns')
    # Run KMeans on these columns
    k = kmeans.fit(dropCol)
    # Add column of KMeans clusters to original dataframe
    df['kMeans'] = k.labels_
    
    # For loop to find mean values for each cluster per column
    for i in range(num):
        # Easier indexing
        c = df.set_index('kMeans')
        # Group by kMeans and find the mean values of each column
        c = c.groupby('kMeans').mean()

        # For each kMeans cluster, add average values from above for all NA values
        # This allows to estimate likely missing values based on their cluster when excluded
        df.loc[df['kMeans'] == i] = df.loc[df['kMeans'] == i].fillna(c.loc[i])
    
    # Reset the index back to normal
    df.set_index(index)
    # Remove old kMeans clusters
    df = df.drop(['kMeans'], axis=1)
    
    # Re-calculate KMeans using new imputed mean values
    k = kmeans.fit(df)
    # Add the new KMeans clusters
    df['kMeans'] = k.labels_
    
    # Return dataframe with KMeans clusters
    return df

In [ ]:
## Join all attributes into one dataframe
infra = [slowBroadband, pTransport]
KM = planning.join(houseSales)
KM = KM.join(infra)

In [ ]:
# Standardise each row with my function defined above
KM_z = standardise(KM)

#### *KMeans Elbow to choose K clusters*

In [ ]:
# have to drop NAs for this method
KM_NA = KM_z.dropna()

## KMeans determine k (optimal number of clusters) code adapted from:
# https://pythonprogramminglanguage.com/kmeans-elbow-method/
distortions = [] # empty list
K = range(1,10) # define range of K (number of clusters)
# iterate through number of clusters
for k in K:
    # choose k clusters for KMeans
    kmeanModel = KMeans(n_clusters=k)
    # fit the model
    kmeanModel.fit(KM_NA)
    # Find the level of distortion using total distance from cluster central points (cdist)
    distortions.append(sum(np.min(cdist(KM_NA, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / KM_NA.shape[0])

# Plot the elbow
## Do not want to display in final document

##plt.plot(K, distortions, 'bx-')
##plt.xlabel('k')
##plt.ylabel('Distortion')
##plt.title('The Elbow Method showing the optimal k')
##plt.show()

#### *K Means Analysis*

In [ ]:
np.random.seed(0) # Make sure my K Means clusters don't change

# Use my function above to run K Means with null values to produce 3 clusters
KM_z = kmeans_withnull(KM_z, 3)
# Add the LSOA geometry
KM_z = KM_z.merge(lsoas, on='LSOA11CD')
# Set the crs as British National Grid
KM_z.crs = {'init': 'epsg:27700'}
# Convert to a GeoDataFrame
KM_z = gpd.GeoDataFrame(KM_z, crs=KM_z.crs, geometry=KM_z.geometry)

In [ ]:
a = [1] # I use an array here in case I want more than 1 cluster

# Define the K Means sub group containing just the cluster of interest
KMS_z = KM_z.loc[KM_z['kMeans'].isin(a)]  

In [ ]:
## Create a hacky border for the chosen cluster
# Change name to avoid altering data
selected_cluster = KMS_z
# Dissolve all polygons into one
selected_cluster = selected_cluster.dissolve(by='kMeans')
# Change back into a geodataframe
selected_cluster = gpd.GeoDataFrame(selected_cluster, crs=lsoas.crs, geometry=selected_cluster.geometry)

#### *Plot K Means*

In [ ]:
# Set ax to Spherical Mercator, required to use the basemap, uses LSOA polys to choose raster image/location
# lsoas not needed to alpha set to 0
ax_bm = lsoas.to_crs(epsg=3857).plot(figsize=(7.5, 9), alpha=0)
# Add the default basemap from Contextily
ctx.add_basemap(ax_bm, alpha=0.5)

# Plot K Means choropleth showing the three clusters, include legend, and custom colour scale
# have to change all crs to project onto basemap
KM = KM_z.to_crs(epsg=3857).plot(ax=ax_bm, column='kMeans', categorical=True, alpha=1, linewidth=.5, legend=True,
          edgecolor='black', cmap='tab20c')

# Add red border to LSOAs from chosen cluster, patheffects used for a shadow effect
selected_cluster.to_crs(epsg=3857).plot(ax=ax_bm, linewidth=1, edgecolor='red', facecolor='none',
                                        path_effects=[path_effects.Stroke(linewidth=3, foreground='black',
                                        alpha=0.4), path_effects.Normal()])

# Change crs of City Centre point to spherical mercator
cityCentre_bm = cityCentre.to_crs(epsg=3857)
# Take x and y of City Centre from geometry
x = cityCentre_bm.geometry.x
y = cityCentre_bm.geometry.y

# Add label for the city centre
text = ax_bm.annotate('City Centre', color='black', xy=(x,y), alpha=1)
# Create a 'buffer' effect for visibility
text.set_path_effects([path_effects.Stroke(linewidth=3, foreground='white', alpha=0.8),
                       path_effects.Normal()])

# Remove axis ticks and labels
ax_bm.set_yticklabels([])
ax_bm.set_xticklabels([])
ax_bm.set_xticks([])
ax_bm.set_yticks([])

# Add a title
plt.title('Figure 1: Geodemographic Classification of Liverpool by Infrastructure and Popularity',
          fontsize=14, weight='bold')
# Display the map
plt.show()

In [ ]:
# Find the mean value for each attribute in each cluster
K3 = KM_z.groupby('kMeans').mean()
# Rename attribute names for clarity
K3 = K3.rename(index=str, columns={"planning": "Planning", "dist_pt": "Distance by Public Transport",
                              "<10mb": "Slow Broadband", "Sales_Change": "House Sales Change"})
# Round to 3 decimals
K3 = np.round(K3, decimals=3)

### Table 2: Average Attribute values by K Means Cluster

In [ ]:
K3.T # Use to choose which cluster(s) to keep

Recent public planning decisions by postcode were stripped from the Liverpool
City Council website and used as a metric to determine the desirability of areas
for regeneration of existing buildings. This, paired with the proportional
increase in house sales between 1995-2005, and 2005-2015 was used to assess the
upcoming popularity of an area (Brendan, 2010)⁠⁠, and where focused improvements
should be targeted.

Public transport and Broadband speed were chosen as
targeted infrastructures, the average time houses in a particular postcode are
able to use public transport and walking to reach their nearest workzone. Public
transport is integral to the ideal of what is considered to be a sustainable
city, where public transport encourages alternative methods of travel of which
there is a minimised environmental impact in comparison to travel purely by car
(Banister, Wood, & Watson, 1997)⁠. Research in the UK makes explicit links
between poverty and transport disadvantage (Lucas, 2012)⁠⁠. Past studies have
put emphasis on the importance of a suitable broadband infrastructure for
enconomic growth (Riddlesden & Singleton, 2014)⁠⁠, and the growth of broadband
in households and industry in the UK has transformed various services and
businesses through the feasibility to work purelly digitally (Caio, 2008)⁠.
Figure 1 shows the clusters determined through a geodemographic analysis based
on key infrastructures using a K Means analysis of the chosen attributes. Three
clusters were chosen through the "K Means Elbow Method" which uses diminishing
returns in distortion to determine the optimal number of clusters (Kodinariya &
Makwana, 2013)⁠.

Cluster 1 (highlighted in red) is chosen to represent LSOAs
which share similar levels of infrastructural disadvantage, but with higher than
average levels of popularity (Table 1).

### DBSCAN to determine crime hotspots

#### *DBSCAN analysis*

In [ ]:
# Obtain the number of points 0.1% of the total represents
minp = np.round(crime.shape[0] * 0.01)

In [ ]:
np.random.seed(0) # Keep same results

# DBSCAN - Density-Based Spatial Clustering of Applications with Noise
# Finds core samples of high crime density and expands clusters from them, radis of 400m
cs, lbls = dbscan(crime[['Easting', 'Northing']], eps=400, min_samples=minp)

# Turn labels into a Series
lbls = pd.Series(lbls, index=crime.index)

# Find core points from DBSCAN
cores = crime.iloc[cs, :]
# Find all noise from DBSCAN
noise = crime.loc[lbls==-1]
# Subtract noise points from all points to find not noise using index difference
not_noise = crime.loc[crime.index.difference(noise.index)]

# Remove columns and set index
not_noise = not_noise.set_index('LSOA11CD')
not_noise = not_noise[['Crime type', 'geometry']]

#### *Plot Setup*

In [ ]:
# Find crime points within chosen K Means clusters
crime_cluster = gpd.sjoin(crime, KMS_z, how='inner')
# Remove index right so I can join again below
crime_cluster = crime_cluster.drop(['index_right'], axis=1)

# Join lsoa polygons to crime_cluster to create DBSCAN Polygons
db_polys = gpd.sjoin(lsoas, crime_cluster, how='inner')
# Drop index_right again
db_polys = db_polys.drop(['index_right'], axis=1)
# Remove all duplicate polygons
db_polys = db_polys[~db_polys.index.duplicated(keep='first')]
# Find non noise points which fall inside the cluster chosen
db_polys = gpd.sjoin(db_polys, not_noise, how='inner')
# Again remove duplicate polygons
db_polys = db_polys[~db_polys.index.duplicated(keep='first')]

In [ ]:
# Create an overall border for the LSOAs
# Call bg to avoid editing lsoas
bg = lsoas
# Add column to dissolve by all polygons
bg['border'] = 1
# Dissolve all polygons into one
bg = bg.dissolve(by='border')

#### *DBSCAN Plot*

In [ ]:
# Set axis to use spherical mercator projection so open source basemap can be used
# chooses the location based on the lsoa polygons
## NOTE: all plots on this axis must use spherical mercator
ax_bm = lsoas.to_crs(epsg=3857).plot(figsize=(7.5, 9), color='white', alpha=0)
# Add default basemap from contextily
ctx.add_basemap(ax_bm, alpha=0.5)

# Plot all chosen K Mean cluster polygons for context (faded)
KMS_z.to_crs(epsg=3857).plot(ax=ax_bm, color='lightgrey', linewidth=0.3, edgecolor='black', alpha=1)


# Plot the K Mean cluster polygons that have crime non noise points inside
db_polys.to_crs(epsg=3857).plot(ax=ax_bm, color='red', linewidth=0.5, edgecolor='black', alpha=0.8)

# Plot DBSCAN noise crime points in grey
noise.to_crs(epsg=3857).plot(ax=ax_bm, markersize=15, color='grey', alpha=0.1)
# Plot DBSCAN not noise points in orange
not_noise.to_crs(epsg=3857).plot(ax=ax_bm, markersize=15, color='orange', alpha=0.1)
# Plot DBSCAN core points in red
cores.to_crs(epsg=3857).plot(ax=ax_bm, markersize=15, color='red', alpha=0.1)


# Add a border to the lsoas, includes some shadowing for aesthetics
bg.to_crs(epsg=3857).plot(ax=ax_bm, color='none', edgecolor='black', linewidth=1,
                          path_effects=[path_effects.Stroke(linewidth=3, foreground='black', alpha=0.4),
                           path_effects.Normal()])

# Change crs of City Centre point to spherical mercator
cityCentre_bm = cityCentre.to_crs(epsg=3857)
# Take x and y of City Centre from geometry
x = cityCentre_bm.geometry.x
y = cityCentre_bm.geometry.y

# Add label for the city centre
text = ax_bm.annotate('City Centre', color='black', xy=(x,y), fontsize=12, alpha=0.8)
# Create a text 'buffer' effect for visibility
text.set_path_effects([path_effects.Stroke(linewidth=3, foreground='white'),
                       path_effects.Normal()])

# Plot then main title
plt.title('Figure 2: DBSCAN Clusters of Crime in Liverpool', fontsize=14, weight='bold')

## INSET PLOT
# Create axes for inset plot
axins = inset_axes(ax_bm, width="25%", height=2.0, loc=3)
# Add LSOA background to inset to show context
inset = lsoas.plot(ax=axins, color='lightgrey', alpha=0.5)
# Add KMeans chosen cluster LSOA in grey
KMS_z.plot(ax=axins, color='lightgrey', linewidth=0, edgecolor='black', alpha=1)
# Show results of DBSCAN LSOA in red
db_polys.plot(ax=axins, linewidth=0.3, edgecolor='red', color='red')

# Remove inset axis ticks and labels
axins.set_yticklabels([])
axins.set_xticklabels([])
axins.set_xticks([])
axins.set_yticks([])
##

# Redimensionate X and Y axes to desired bounds
ax_bm.set_ylim(7051000, 7069000)
ax_bm.set_xlim(-337500, -324000) # dont set to higher than -337500

# Remove axis ticks and labels
ax_bm.set_yticklabels([])
ax_bm.set_xticklabels([])
ax_bm.set_xticks([])
ax_bm.set_yticks([])

# Display the map
plt.show()

Clusters of high crime are identified in Figure 2 with the use of DBSCAN, this
technique clusters crime points that are spatially nearby, shown in red and
orange, and considers points with few neighbours to be noise (Grey points;
Figure 2). LSOA polygons from the chosen K Means cluster that contain one or
more clustered points (non noise) are highlighted in red.

Urban regeneration
has shown to positively impact areas with lower socioeconomic status and health
(Thomson et al., 2006)⁠, where typically there are high levels of crime (Steptoe
& Feldman, 2001)⁠. Crime is concentrated towards the city centre, however in a
tourist city like Liverpool this is due to the tourist population primarily and
less due to the underlying population (Calafat et al., 2011; Hughes et al.,
2008).

### Conclusion

#### *Select Final 5*

In [ ]:
# Find top 5 polygons with worst infrastructure, longest time to work zone by public transport
# and slowest broadband speeds
Chosen = db_polys.sort_values(['dist_pt', '<10mb'], ascending=[False, False]).head(5)

In [ ]:
# Subset table to show only columns of interest
ChosenTable = Chosen[['planning', 'dist_pt', '<10mb', 'Sales_Change']]

#### *Plot Setup*

In [ ]:
# Rename bb (broadband) for KDE to ensure it isn't overwritten
kdebb = bb
# Append column for postcodes with over 50% of homes with less than 10mb download
kdebb['slow'] = kdebb['<10mb'] > 50
# Use boolean values that =True when >50% of homes within a postcode
# have less than 10mb download speeds
kdebb['slow'] = kdebb['slow'].astype(int)
# Subset into just slow postcodes for KDE plot to show black spots
kdebb = kdebb.loc[kdebb['slow'] == 1]

In [ ]:
# Buffer bg to create a clean edge for KDE
bg = bg.buffer(10000)
# Change back into a geodataframe
bg = gpd.GeoDataFrame(bg, crs=lsoas.crs, geometry=bg.geometry)
# Overlay set difference to remove bg that overlaps the lsoas
bg = gpd.overlay(bg, lsoas, how='difference')

In [ ]:
# Append broadband geometry for KDE plot
kdebb['X'] = kdebb.geometry.x
kdebb['Y'] = kdebb.geometry.y

#### *DBSCAN Plot*

In [ ]:
# Setup figure and main ax
f, ax = plt.subplots(1, figsize=(9, 9))

# Plot LSOA background for colour/context
lsoas.plot(ax=ax, color='lightgrey', alpha=0.5)

# Plot LSOAS greyed out that are removed from this analysis
db_polys.plot(ax=ax, color='lightgrey', linewidth=0, edgecolor='black', alpha=1)
# Plot broadband KDE underlying LSOAs showing black spots in dark red
sns.kdeplot(kdebb['X'], kdebb['Y'], ax=ax, n_levels=20, shade=True, cmap='Reds', alpha=0.3,
           shade_lowest=False)
# Plot choropleth of 5 chosen LSOAs
Chosen.plot(ax=ax, column = Chosen['LSOA11CD'], alpha = 0.8, linewidth=1,
            legend = True, edgecolor='black')

## Move legend so it doesn't overlap polygon
leg = ax.get_legend()
leg.set_bbox_to_anchor((0., 0.67, 0.3, 0.3))

# Make sure KDE doesn't appear outside MSOA boundary, uses overlay difference
bg.plot(ax=ax, color='#f6f6f6', edgecolor='black')

# Find x y geometry for city centre point to add label
x = cityCentre.geometry.x
y = cityCentre.geometry.y

# Add label for the city centre
text = ax.annotate('City Centre', color='black', xy=(x, y))
# Create a text 'buffer' effect for visibility
text.set_path_effects([path_effects.Stroke(linewidth=3, foreground='white'),
                       path_effects.Normal()])

# Set main plot title
plt.title('Figure 3: Chosen 5 LSOAs showing Broadband "Black spots"', fontsize=14, weight='bold')

## INSET PLOT
# Add an axis for an inset plot, gives context to zoomed plot
axins = inset_axes(ax, width="25%", height=2.0, loc=3)
# Add LSOA background to inset
inset = lsoas.plot(ax=axins, color='lightgrey', alpha=0.5)

# Show faded removed LSOAs from DBSCAN
db_polys.plot(ax=axins, color='lightgrey', linewidth=0, edgecolor='black', alpha=1)
# Show chosen LSOAs on inset
Chosen.plot(ax=axins, linewidth=0.3, edgecolor='red', color='red')

# Remove inset axis labels and ticks
axins.set_yticklabels([])
axins.set_xticklabels([])
axins.set_xticks([])
axins.set_yticks([])
##

# Keep axes proportionate
plt.axis('equal')

# Redimensionate X and Y axes to desired bounds
ax.set_ylim(386000, 397000)
ax.set_xlim(330000, 339000)

# Remove remove axis labels and ticks (not sure why Y and X showing)
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])

# Draw map
plt.show()

In [ ]:
# Rename attributes for clarity
ChosenTable = ChosenTable.rename(index=str, columns={"planning": "Planning",
                                                     "dist_pt": "Distance by Public Transport",
                                                     "<10mb": "Slow Broadband",
                                                     "Sales_Change": "House Sales Change"})

### Table 2: Chosen LSOAs showing all standardised attributes

In [ ]:
ChosenTable.T # Show table with 5 LSOAS and their standardised varibles (transposed)

With 7 LSOAs left from the DBSCAN analysis, the final 5 are chosen based on the
highest levels of need for infrastructural improvement. With this, Figure 3
displays the areas in which Broadband is the slowest with a Kernal Density
Estimate plot, deemed ‘black spots’, with a higher number of properties with
slower broadband in darker red. LSOAs not within black spots, and further from
the city centre typically are in need of public transport improvement.

Table 2
gives a list of the chosen LSOAs with their repective standardised attributes,
showing the standard deviation in relation to the mean average of all LSOAs.

## References

Acock, A. C. (2005). Working with missing values. Journal of Marriage and
Family. https://doi.org/10.1111/j.1741-3737.2005.00191.x

Banister, Wood, &
Watson. (1997). Sustainable cities, transport, energy and urban form,
Environment and planning,. Final Report for the URBAN 21 Project, 24,
pp.125-143.

Brendan, N. (2010). Housing Market Renewal in Liverpool: Locating
the gentrification debate in history, context and evidence. Housing Studies,
25(5), 715–733. https://doi.org/10.1080/02673037.2010.483587

Caio, F. (2008).
The Next Phase of Broadband UK: Action now for long term competitiveness - Final
Report, (September).

Calafat, A., Blay, N., Bellis, M., Hughes, K., Kokkevi,
A., Mendes, F., … Tripodi, S. (2011). Tourism, nightlife and violence: a cross
cultural analysis and prevention recommendations. Retrieved from
http://www.irefrea.eu/uploads/PDF/Calafatetal_2010.pdf

Hughes, K., Anderson,
Z., Morleo, M., & Bellis, M. A. (2008). Alcohol, nightlife and violence: The
relative contributions of drinking before and during nights out to negative
health and criminal justice outcomes. Addiction.
https://doi.org/10.1111/j.1360-0443.2007.02030.x

Kodinariya, T. M., & Makwana,
P. R. (2013). Review on determining number of Cluster in K-Means Clustering.
International Journal of Advance Research in Computer Science and Management
Studies, 1(6), 2321–7782. https://doi.org/10.1109/PDP.2013.84

Lucas, K. (2012).
Transport and social exclusion: Where are we now? Transport Policy, 20, 105–113.
https://doi.org/10.1016/j.tranpol.2012.01.013

Mesquita, D. P. P., Gomes, J. P.
P., & Rodrigues, L. R. (2016). K-means for datasets with missing attributes:
Building soft constraints with observed and imputed values. ESANN 2016 - 24th
European Symposium on Artificial Neural Networks, (April), 27–29.

Mohamad, I.
Bin, & Usman, D. (2013). Standardization and its effects on K-means clustering
algorithm. Research Journal of Applied Sciences, Engineering and Technology,
6(17), 3299–3303. https://doi.org/10.19026/rjaset.6.3638

Riddlesden, D., &
Singleton, A. D. (2014). Broadband speed equity: A new digital divide? Applied
Geography, 52, 25–33. https://doi.org/10.1016/j.apgeog.2014.04.008

Steptoe, A.,
& Feldman, P. J. (2001). Neighborhood problems as sources of chronic stress:
Development of a measure of neighborhood problems, and associations with
socioeconomic status and health. Annals of Behavioral Medicine, 23(3), 177–185.
https://doi.org/10.1207/S15324796ABM2303_5

Thomson, H., Atkinson, R.,
Petticrew, M., & Kearns, A. (2006). Do urban regeneration programmes improve
public health and reduce health inequalities? A synthesis of the evidence from
UK policy and practice (1980-2004). Journal of Epidemiology and Community
Health, 60(2), 108–115. https://doi.org/10.1136/jech.2005.038885

## Data Sources

Map tiles by Stamen Design, under CC BY 3.0. Data by OpenStreetMap, under ODbL.
Department for Transport. (2013). Travel time, destination and origin indicators
to key sites and services, by Lower Super Output Area (LSOA) (ACS05). Retrieved
from https://www.gov.uk/government/statistical-data-sets/acs05-travel-time-
destination-and-origin-indicators-to-key-sites-and-services-by-lower-super-
output-area-lsoa

Home Office. (2018). data.police.uk. Retrieved from
https://data.police.uk/

Liverpool City Council. (2018). Recent Decisions.
Retrieved from
http://northgate.liverpool.gov.uk/PlanningExplorer17/RecentDecisionsSearch.aspx
Ordnance Survey. (2018). Code-Point Open. Retrieved from
https://www.ordnancesurvey.co.uk/business-and-government/products/code-point-
open.html

Singleton, A., & Hai, N. (2015). CDRC 2011 OAC Geodata Pack -
Liverpool (E08000012). Retrieved from
https://data.cdrc.ac.uk/dataset/cdrc-2011-oac-geodata-pack-liverpool-e08000012
Singleton, A., Pavlis, M., & University of Liverpool. (2014). CDRC 2014 Fixed
Broadband Geodata Pack. Retrieved from
https://data.cdrc.ac.uk/dataset/cdrc-2014-fixed-broadband-geodata-pack-
liverpool-e08000012

Singleton, A., Pavlis, M., & University of Liverpool.
(2015). CDRC Median House Prices Geodata Pack (1995-2015). Retrieved from
https://data.cdrc.ac.uk/dataset/cdrc-median-house-prices-geodata-
pack-1995-2015-liverpool-e08000012